In [1]:
%pwd

'd:\\Projects\\kidney-disease-classification\\research'

In [2]:
import os
os.chdir('../')
%pwd

'd:\\Projects\\kidney-disease-classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [15]:
class configurationManager:
    def __init__(self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])


        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(
                prepare_base_model.updated_base_model_path
            ),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGUMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [13]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        datagen_kwargs = dict(rescale=1./255, validation_split=0.20)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

        self.valid_generator = valid_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                shear_range=0.2,
                width_shift_range=0.2,
                height_shift_range=0.2,
                zoom_range=0.2,
                **datagen_kwargs
            )
        else:
            train_datagen = valid_datagen

        self.train_generator = train_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(model: tf.keras.Model, model_path: Path):
        model.save(model_path)


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
        )

        self.save_model(
            model=self.model,
            model_path=self.config.trained_model_path
        )

    

In [19]:
try:
    config = configurationManager()
    training_config = config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

[2025-12-04 15:45:33,349: INFO: common: yaml file: configs\config.yaml loaded successfully]
[2025-12-04 15:45:33,351: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-04 15:45:33,352: INFO: common: Directory created at: artifacts]
[2025-12-04 15:45:33,354: INFO: common: Directory created at: artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/50
23/23 [==============================] - 92s 4s/step - loss: 11.5155 - accuracy: 0.5309 - val_loss: 10.7981 - val_accuracy: 0.4375
Epoch 2/50
23/23 [==============================] - 82s 4s/step - loss: 5.7645 - accuracy: 0.7303 - val_loss: 8.9906 - val_accuracy: 0.6000
Epoch 3/50
23/23 [==============================] - 91s 4s/step - loss: 6.7813 - accuracy: 0.7079 - val_loss: 8.9152 - val_accuracy: 0.4375
Epoch 4/50
23/23 [==============================] - 89s 4s/step - loss: 2.6719 - accuracy: 0.7978 - val_loss: 0.6969 - val_accuracy: 0.9500
Epoch 5/50
23/23 [========